# Convert token to token ID
- 이전 코드에서 Edith Wharton의 단편 소설을 개별 토큰으로 분할하였다.
- 이번에는 이러한 토큰을 Python 문자열에서 정수 표현으로 변환하여 "토큰 ID"를 생성할 것이다.
- 이전에 생성한 토큰을 토큰 ID로 매핑하려면 먼저 소위 **어휘(vocabulary)**를 만들어야 한다.
    - 이 어휘는 각 고유 단어와 특수 문자를 고유한 정수에 매핑하는 방법을 정의한다.

<img src="img/3.png" alt="OLS">

- 어휘를 구축하기 위해 훈련 데이터셋의 전체 텍스트를 개별 토큰으로 토큰화한다.
- 이 개별 토큰을 알파벳순으로 정렬하고 중복된 토큰을 제거한다.
- 고유 토큰을 모아 어휘를 생성하며, 이는 각 고유 토큰을 고유한 정수 값에 매핑하는 방법을 정의한다.

In [2]:
import re

with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]

In [ ]:
# set()을 통해 중복 제거 & sorted()로 알파벳 순으로 정렬
all_words = sorted(set(preprocessed))
vocab_size = len(all_words) # 고유 단어 리스트의 길이
print(vocab_size)

1130


In [ ]:
# vocab dictionary 생성 : enumerate(all_words) -> (index, element) 이터레이터 반환
vocab = {token:integer for integer,token in enumerate(all_words)}

# vocab.items() -> 딕셔너리의 key-value쌍을 (key, value) 형태로 꺼냄
for i, item in enumerate(vocab.items()):
    print(item)
    if i > 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)
('His', 51)


***
- 위의 출력 결과를 보면, 각 고유한 정수 라벨과 연결된 개별 토큰이 포함된 dictionary를 확인할 수 있다.
- 다음 목표는 이 어휘를 사용하여 새 텍스트를 토큰 ID로 변환하는 것이다!

<img src="img/4.png">

- 텍스트를 토큰화하고, 문자열을 정수로 매핑하여 어휘를 통해 토큰 ID를 생성하는 encode 메서드를 포함하는 간단한 파이썬 토크나이저 클래스를 구현해보자.

In [ ]:
# 심플 텍스트 토크나이저의 구현

class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = ' '.join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

1. 생성자
- SimpleTokenizerV1 클래스의 생성자에서 vocab을 받는 이유는 이후 encode()와 decode() 함수에서 사용할 단어 <=> ID 매핑 정보를 저장하기 위해서이다.

- self.str_to_int:
    - 단어 -> 숫자 ID

- self.int_to_str:
    - 숫자 ID -> 단어

***

2. encode
- 텍스트가 들어왔을 때, 공백과 특수문자를 기준으로 분할하여 전처리하고
- 전처리된 리스트 preprocessed를 통해 str_to_int[s]로 인덱싱하여 토큰의 고유 ID를 갖는 리스트 ids를 반환한다.

***

3. decode
- 고유 ID 리스트 ids를 토대로 각 ID에 해당하는 토큰 조회
- 공백으로 단어들을 이어붙힌다.
- 구두점 앞 공백을 제거하고
- 최종 텍스트를 반환한다.

> 위 과정을 그림으로 요약한 것이 밑의 그림이다.

<img src="img/5.png">

In [6]:
# 새로운 토크나이저 객체 생성
tokenizer = SimpleTokenizerV1(vocab)
text = """"It's the last he painted, you know," Mrs. Gisburn said with pardonable
pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [7]:
# 위의 토큰 ID를 decode를 통해 텍스트로 다시 변환
print(tokenizer.decode(ids))

" It ' s the last he painted , you know , " Mrs . Gisburn said with pardonable pride .
